# Bibliotecas

In [1]:
import random
import re
import nltk
import unidecode
import math
import time
import numpy as np
import pandas as pd
import seaborn as sns
from numpy import mean
from nltk import tokenize
from string import punctuation
from string import ascii_letters
from nltk.stem import RSLPStemmer
from sklearn import metrics
from pandas_ods_reader import read_ods
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_curve
from sklearn.multiclass import OneVsRestClassifier
from sklearn.calibration import CalibratedClassifierCV
from pylab import plot, show, savefig, xlim, figure, ylim, legend, boxplot, setp, axes
from matplotlib import pyplot as plt
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('rslp')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/gabrielrucker/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/gabrielrucker/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/gabrielrucker/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     /home/gabrielrucker/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

# Funções

In [2]:
#Função para ler os dados e realizar algumas definições
#Entrada: Nenhuma
#Saída: Dataframe com os dados e os tipos atribuídos

def leitura_e_configuracao():
    df = read_ods("base-de-dados-cientificos.ods",1)

    #Define os tipos de dados das colunas
    df['Palavras-chave'] = df['Palavras-chave'].astype(str)
    df['Título'] = df['Título'].astype(str)
    df['Resumo'] = df['Resumo'].astype(str)
    df['Categoria'] = df['Categoria'].astype(str)
    df['ID'] = df['ID'].astype(int)
    
    # atribuição dos ids a cada uma das categorias
    for abstract in df:
        df.loc[df['Categoria'] == 'Ciências Agrárias', 'ID_Categoria'] = 0
        df.loc[df['Categoria'] == 'Ciências Biológicas', 'ID_Categoria'] = 1
        df.loc[df['Categoria'] == 'Ciências Exatas e da Terra', 'ID_Categoria'] = 2
        df.loc[df['Categoria'] == 'Ciências Humanas', 'ID_Categoria'] = 3
        df.loc[df['Categoria'] == 'Ciências Sociais Aplicadas', 'ID_Categoria'] = 4
        df.loc[df['Categoria'] == 'Ciências da Saúde', 'ID_Categoria'] = 5
        df.loc[df['Categoria'] == 'Engenharias', 'ID_Categoria'] = 6
        df.loc[df['Categoria'] == 'Lingüística, Letras e Artes', 'ID_Categoria'] = 7
        df.loc[df['Categoria'] == 'Multidisciplinar', 'ID_Categoria'] = 8
    
    df['ID_Categoria'] = df['ID_Categoria'].astype(int) #tipo de dado da nova coluna criada
    df['Texto'] = df['Título'] + df['Resumo'] + df['Palavras-chave'] # Atributo texto é a união do título, resumo
                                                                     # e palavras-chave
    
    return df # retorna o dataframe
    
#Função para realizar o pré-processamento dos dados
#Entrada: Dataframe com os dados e o atributo a ser pré-processado
#Saída: Nenhuma

def pre_processamento(df, atributo):
    
    #letras minúsculas
    df[atributo] = [texto.lower() for texto in df[atributo]]

    #remoção de acentos
    df[atributo] = [unidecode.unidecode(texto) for texto in df[atributo]]

    #remoção de stop words
    stop_words = nltk.corpus.stopwords.words('portuguese')
    stop_words.extend(['artigo','sobre', 'resultado','objetivo','estudo','brasil', 'trabalho', 
                      'delineamento', 'experimental','analise'])
    lista_stopwords = []
    for abstract in df[atributo]:
        tokens = tokenize.WhitespaceTokenizer().tokenize(abstract)
        novo_texto = ' '.join([token for token in tokens if token not in stop_words])
        lista_stopwords.append(novo_texto)
    df[atributo] = lista_stopwords

    #remoção de pontuação
    pontuacoes = list(punctuation)
    pontuacao = []
    for texto in df[atributo]:
        tokens = tokenize.WhitespaceTokenizer().tokenize(texto)
        novo_texto = ' '.join([token for token in tokens if token not in pontuacoes])
        pontuacao.append(novo_texto)
    df[atributo] = pontuacao

    #Retira textos não desejados
    df[atributo] = df[atributo].apply(lambda x: re.sub(r'[0-9]','',x))
    df[atributo] = df[atributo].apply(lambda x: re.sub(r'[/(){}\[\]\#\|@,;.:-]',' ',x))
    df[atributo] = df[atributo].apply(lambda x: re.sub(r'\d+', '', x))

    #obtenção da raíz das palavras
    stemmer = RSLPStemmer()
    raizes = []
    for texto in df[atributo]:
        tokens = tokenize.WhitespaceTokenizer().tokenize(texto)
        novo_texto = ' '.join([stemmer.stem(token) for token in tokens])
        raizes.append(novo_texto)
    df[atributo] = raizes
    

def extracao_caracteristicas(df, atributo):
    # aplicação do TF-IDF para extração de características
    tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', ngram_range=(1, 2))
    vetor_caracteristicas = tfidf.fit_transform(df[atributo]).toarray()
    rotulos = df.ID_Categoria
    return vetor_caracteristicas, rotulos

def selecao_caracteristicas(vetor_caracteristicas, rotulos):
    chi2_test = SelectKBest(chi2,k=int((vetor_caracteristicas.shape[1]/2)))
    vetor_caracteristicas = chi2_test.fit_transform(vetor_caracteristicas,rotulos)
    return vetor_caracteristicas

def classificacao(df, modelo, vetor_caracteristicas, rotulos):
    
    ### cross-validation ###
    
    vc_df = pd.DataFrame(index=range(10))
    entradas = []
    acuracias = cross_val_score(modelo, vetor_caracteristicas, rotulos, scoring='accuracy', cv=10)
    for num_fold, acuracia in enumerate(acuracias):
        entradas.append((num_fold, acuracia))
    vc_df = pd.DataFrame(entradas, columns=['num_fold', 'acuracia'])
    
    return vc_df.acuracia.mean() # média das acurácias

def cria_grafico(dimensao, atributo, classificador):
    ruidos = [0, 10, 20, 30, 40]
    acuracias = []
    for ruido in ruidos:
        df = leitura_e_configuracao()
        if(dimensao == 'Acurácia-add'):
            contamina_acuracia(df, atributo, 'adicao', ruido)
        elif(dimensao == 'Acurácia-rm'):
            contamina_acuracia(df, atributo, 'remocao', ruido)
        elif(dimensao == 'Acurácia-subs'):
            contamina_acuracia(df, atributo, 'substituicao', ruido)
        elif(dimensao == 'Completude'):
            contamina_completude(df, atributo, ruido)
        elif(dimensao == 'Consistência'):
            contamina_consistencia(df, atributo, ruido)
        pre_processamento(df, atributo)
        vetor_caracteristicas, rotulos = extracao_caracteristicas(df, atributo)
        vetor_caracteristicas = selecao_caracteristicas(vetor_caracteristicas, rotulos)
        acuracia = classificacao(df, classificador, vetor_caracteristicas, rotulos)
        acuracias.append(acuracia)
    plt.show()
    
    return acuracias

In [3]:
def gera_lista_valores(string):
    lista_valores = []
    lista_carac = []
    i = 0
    for i in range(len(string)):
        lista_carac.append(string[i])
    return [int(ord(carac)) for carac in lista_carac]

def calcula_media(lista_valores):
    soma = 0
    media = 0
    for valor in lista_valores:
        soma += valor
    return soma / len(lista_valores)

def calcula_desvio(lista_valores, media):
    desvio = 0
    for valor in lista_valores:
        desvio += (valor - media)**2
    return (desvio / len(lista_valores))**(1/2)

def ruido_gaussiano(media, desvio, valor):
    return (1 / (desvio * math.sqrt(2 * math.pi))) * math.exp(-(((valor - media) ** 2) / (2 * (desvio ** 2))))

In [4]:
def contamina_acuracia(df, escolha_atributo, metodo, porc_ruido):
    lista_artigos = []
    
    if (porc_ruido == 0): # retorna caso não haja contaminação
        return
    
    qtd_artigos = int(df.shape[0] * (porc_ruido/100)) # quantidade de artigos que serão contaminados
    atributo_contaminado = escolha_atributo 
    i = 0
    
    while i < qtd_artigos: # laço para contaminar os artigos
        
        lista_carac = []
        lista_valores = []
        indice = random.choice(df.index) # artigo é selecionado de maneira aleatória
        
        if indice in lista_artigos: # verifica se o artigo já foi contaminado previamente
            while indice in lista_artigos:
                indice = random.choice(df.index)
                
        lista_valores = gera_lista_valores(df.loc[indice, atributo_contaminado]) # obtém os valores em ascii 
                                                                                 # em formato de lista
        media = calcula_media(lista_valores) # cálculo da média dos valores em ascii
        desvio = calcula_desvio(lista_valores, media) # desvio padrão dos valores em ascii
        
        for valor in lista_valores: # percorre os caracteres em ascii
            
            if (((0.607 / (math.sqrt(2*math.pi)*desvio)) <= ruido_gaussiano(media,desvio,valor) <= 1) 
                and (chr(valor).isalpha())): # se o caracter for contaminado
                
                caracter = random.choice(ascii_letters) # obtém um caracter aleatório
                
                if(not caracter.isalpha()): # gera novo caracter enquanto este não for alfabético
                    while(not caracter.isalpha()):
                        caracter = random.choice(ascii_letters)
                                
                if(metodo == 'substituicao'): # método de substituição de caracteres
                    lista_carac.append(caracter) # adiciona o caracter aleatório no lugar do antigo na nova lista
                elif(metodo == 'adicao'): # método de adição de caracteres
                    lista_carac.extend([chr(valor), caracter]) # adiciona o caracter antigo junto com o aleatório
                                                               # à nova lista
                elif(metodo == 'remocao'): # remoção de caracteres
                    pass # caso o caracter seja contaminado, ele não é adicionado à lista de caracteres
            
            else: # caracter não será contaminado
                lista_carac.append(chr(valor)) # troca o valor em ascii pelo caracter correspondente
                                               # e o adiciona à lista
                
        df.loc[indice, atributo_contaminado] = ''.join([carac for carac in lista_carac]) # substitui o artigo pela
                                                                                         # lista resultante
        lista_artigos.append(indice) # adiciona o índice do artigo à lista de artigos contaminados
        i+=1

def contamina_completude(df, escolha_atributo, porc_ruido):
    lista_artigos = []
    #lista_atributos = ['Título', 'Resumo', 'Palavras-chave']
    if(porc_ruido == 0):
        return
    qtd_artigos = int(df.shape[0] * (porc_ruido/100))
    i = 0
    atributo_contaminado = escolha_atributo
    while i < qtd_artigos:
        indice = random.choice(df.index) # sorteia um artigo aleatório
        if indice in lista_artigos: # se o artigo já foi contaminado
            while indice in lista_artigos: # procura por um novo artigo enquanto estiver na lista dos que já foram contaminados
                indice = random.choice(df.index)
        df[atributo_contaminado] = df[atributo_contaminado].replace([df[atributo_contaminado][indice]], '') # apaga o atributo
        lista_artigos.append(indice) # adiciona o artigo contaminado à lista de artigos que já foram contaminados
        i += 1
        
def contamina_consistencia(df, escolha_atributo1, porc_ruido):
    lista_artigos = []
    lista_atributos = ['Título', 'Resumo', 'Palavras-chave']
    if(porc_ruido == 0):
        return
    porc_ruido /= 2
    qtd_artigos = int(df.shape[0] * (porc_ruido/100)) # número de artigos a serem contaminados
    i = 0
    atributo_contaminado1 = escolha_atributo1
    atributo_contaminado2 = random.choice(lista_atributos)
    while i < qtd_artigos:
        indice1 = random.choice(df.index)
        indice2 = random.choice(df.index)
        if(df['ID_Categoria'][indice1] == df['ID_Categoria'][indice2]):
                while(df['ID_Categoria'][indice1] == df['ID_Categoria'][indice2]):
                        indice1 = random.choice(df.index)
        
        else:
            temp = df[atributo_contaminado1][indice1]
            if(atributo_contaminado2 == atributo_contaminado1):
                while(atributo_contaminado2 == atributo_contaminado1):
                    atributo_contaminado2 = random.choice(lista_atributos)
            df[atributo_contaminado1] = df[atributo_contaminado1].replace([df[atributo_contaminado1][indice1]], [df[atributo_contaminado2][indice2]])
            df[atributo_contaminado2] = df[atributo_contaminado2].replace([df[atributo_contaminado2][indice2]], temp)
            i+=1

In [5]:
def melhor_divisao(modelo):
    lista_acuracias = []
    lista_porc = []
    i = 0.01
    while i < 1:
        X_train, X_test, y_train, y_test = train_test_split(vetor_caracteristicas, rotulos, train_size=i, random_state=0)
        modelo.fit(X_train, y_train)
        y_pred = modelo.predict(X_test)
        lista_acuracias.append(accuracy_score(y_test, y_pred)*100)
        lista_porc.append(i*100)
        i += 0.01
    fig, ax = plt.subplots()
    ax.plot(lista_porc, lista_acuracias, 'black')
    plt.xlabel('Conjunto de Treino (%)')
    plt.ylabel('Acurácia (%)')
    indice = lista_acuracias.index(max(lista_acuracias))
    plt.scatter(lista_porc[indice], max(lista_acuracias), c='red', linewidths=2)
    plt.axhline(y=max(lista_acuracias), xmax=lista_porc[indice]/105, linestyle='--', color="red")
    plt.axvline(x=lista_porc[indice], ymax=max(lista_acuracias)/(max(lista_acuracias)+5), linestyle='--', color="red")
    plt.text(lista_porc[indice]+2, max(lista_acuracias)+0.5, '({:.1f}, {:.1f})'.format(lista_porc[indice], max(lista_acuracias)))
    plt.margins(0.1)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    #font = {'family':'sans-serif',
    #        'weight':'normal',
    #        'size': 16}
    #plt.rc('font', **font)
    
    plt.show()

In [6]:
def gera_curvas_roc(modelo, cjto_treino):

    X_train, X_test, y_train, y_test = train_test_split(vetor_caracteristicas, rotulos, train_size=(cjto_treino/100), random_state=0)
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    pred_prob = modelo.predict_proba(X_test)
    
    FPR = {}
    TPR = {}
    limite = {}

    num_classes = 9

    for i in range(num_classes):
        FPR[i], TPR[i], limite[i] = roc_curve(y_test, pred_prob[:, i], pos_label=i)  
    
    # plotando a área sob a curva
    fig, ax = plt.subplots()
    plt.plot(FPR[0], TPR[0], color='#e41a1c', linestyle='-', label='CA vs Resto', linewidth=3)
    plt.plot(FPR[1], TPR[1], color='#377eb8', linestyle='-', label='CB vs Resto',linewidth=3)
    plt.plot(FPR[2], TPR[2], color='#4daf4a', linestyle='-', label='CET vs Resto', linewidth=3)
    plt.plot(FPR[3], TPR[3], color='#984ea3', linestyle='-', label='CH vs Resto', linewidth=3)
    plt.plot(FPR[4], TPR[4], color='#ff7f00', linestyle='-', label='CSA vs Resto', linewidth=3)
    plt.plot(FPR[5], TPR[5], color='#ffff33', linestyle='-', label='CS vs Resto', linewidth=3)
    plt.plot(FPR[6], TPR[6], color='#a65628', linestyle='-', label='E vs Resto', linewidth=3)
    plt.plot(FPR[7], TPR[7], color='#f781bf', linestyle='-', label='LLA vs Resto', linewidth=3)
    plt.plot(FPR[8], TPR[8], color='#999999', linestyle='-', label='M vs Resto', linewidth=3)
    plt.xlabel('Taxa de Falsos Positivos (1 - Especificidade)', weight='book')
    plt.ylabel('Taxa de Verdadeiros Positivos (Sensibilidade)', weight='book')
    ax.xaxis.set_tick_params(width=2)
    ax.yaxis.set_tick_params(width=2)
    leg = plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0., fontsize=26, shadow=True,
                     fancybox=True)
    leg.get_frame().set_linewidth(3)
    for i in range(num_classes):
        leg.get_lines()[i].set_linewidth(5)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    plt.setp(ax.spines.values(), linewidth=2)
    fig.set_size_inches(20, 10)
    
    font = {'family':'sans-serif',
            'weight':'book',
            'size': 26}
    plt.rc('font', **font)
    
    plt.show()

In [7]:
def cria_box_plot(eixo_x, eixo_y, dados):

    fig, ax = plt.subplots()

    flierprops = dict(marker='*', markerfacecolor='black', markersize=8,
                  markeredgecolor='none')
    medianprops = dict(linestyle='-', linewidth=1.1)

    sns.boxplot(x=eixo_x, y=eixo_y, palette='bright', data=dados, orient='v', width=0.3,
           saturation=1, linewidth=1, showcaps=False, flierprops=flierprops, medianprops=medianprops)
    #sns.stripplot(x='Acurácia (%)', y='Classificador', data=vc_df, 
              #size=10, jitter=0.3, edgecolor="gray", linewidth=1, palette='pastel')

    #plt.xticks(
    #    rotation=45, 
    #    horizontalalignment='right',
    #    fontweight='light',
    #    fontsize='medium'  
    #)

    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

    plt.show()

# Testes-Contaminação

In [ ]:
%%time
atributo = 'Texto'
metodos = ['Acurácia-add','Completude','Consistência']
modelo = LogisticRegression(random_state=0, solver='liblinear')

for metodo in metodos:
    arquivo = open("lr-novo/{}-{}.txt".format(atributo,metodo),"w")
    i=0
    for i in range(10):
        lista_acuracias = cria_grafico(metodo, atributo, modelo)
        for acuracia in lista_acuracias:
            arquivo.write("{} \t".format(acuracia))    
        arquivo.write("\n\n")
    arquivo.close()
    print("{}: {} ✔\n".format(atributo, metodo))

# Pré-processamento/Extração/Seleção

In [ ]:
df = leitura_e_configuracao()

In [ ]:
pre_processamento(df, 'Texto')

In [ ]:
vetor_caracteristicas, rotulos = extracao_caracteristicas(df, 'Texto')

In [ ]:
vetor_caracteristicas = selecao_caracteristicas(vetor_caracteristicas, rotulos)

# Validação

## Validação cruzada

In [ ]:
modelos = [
    LinearSVC(random_state=0),
    MultinomialNB(),
    LogisticRegression(random_state=0, solver='liblinear')
]
    
vc_df = pd.DataFrame(index=range(10 * len(modelos)))

entradas = []

for modelo in modelos:
    nome_modelo = modelo.__class__.__name__
    if(nome_modelo == "LinearSVC"):
        nome_modelo = "MVS"
    elif(nome_modelo == "MultinomialNB"):
        nome_modelo = "NB"
    elif(nome_modelo == "LogisticRegression"):
        nome_modelo = "RL"
    acuracias = cross_val_score(modelo, vetor_caracteristicas, rotulos, scoring='accuracy', cv=10)
    for num_fold, acuracia in enumerate(acuracias):
        entradas.append((nome_modelo, num_fold, acuracia))
        
vc_df = pd.DataFrame(entradas, columns=['Classificador', 'Num_Fold', 'Acurácia (%)'])
vc_df['Acurácia (%)'] = [acuracia * 100 for acuracia in vc_df['Acurácia (%)']]
    
cria_box_plot('Classificador', 'Acurácia (%)', vc_df)

In [ ]:
vc_df['Acurácia'] = vc_df['Acurácia (%)']
acuracias = vc_df.groupby('Classificador').Acurácia.mean()
acuracias.sort_values(ascending=False)

In [ ]:
acuracias_desvios = vc_df.groupby('Classificador').Acurácia.std()
acuracias_desvios.sort_values(ascending=False)

In [ ]:
%%time
modelo = MultinomialNB()
acuracias = cross_val_score(modelo, vetor_caracteristicas, rotulos, scoring='accuracy', cv=10)
print(mean(acuracias)*100)

## Leave-one-out

In [ ]:
%%time
loo = LeaveOneOut()
modelo = LogisticRegression(random_state=0, solver='liblinear')
acuracia = cross_val_score(modelo, vetor_caracteristicas, rotulos, scoring='accuracy', cv=loo)
print(mean(acuracia)*100)

## Divisão treino/teste

In [ ]:
melhor_divisao(LinearSVC(random_state=0))

In [ ]:
%%time
X_train, X_test, y_train, y_test = train_test_split(vetor_caracteristicas, rotulos, train_size=0.63, random_state=0)
modelo = LinearSVC()
modelo.fit(X_train, y_train)
y_pred = modelo.predict(X_test)
print(accuracy_score(y_test, y_pred)*100)

## Matriz de confusão

In [ ]:
modelo = SVC(kernel="linear", probability=True)


X_train, X_test, y_train, y_test = train_test_split(vetor_caracteristicas, rotulos, train_size=0.63, random_state=0)
modelo.fit(X_train, y_train)
y_pred = modelo.predict(X_test)

#criação de dicionário (categoria,id)
category_id_df = df[['Categoria', 'ID_Categoria']].drop_duplicates().sort_values('ID_Categoria')
category_to_id = dict(category_id_df.values)
#criação de dicionário (id, categoria)
id_to_category = dict(category_id_df[['ID_Categoria', 'Categoria']].values)

#criação das siglas
category_id_df.loc[category_id_df['Categoria'] == 'Ciências Agrárias', 'Categoria'] = 'CA'
category_id_df.loc[category_id_df['Categoria'] == 'Ciências Biológicas', 'Categoria'] = 'CB'
category_id_df.loc[category_id_df['Categoria'] == 'Ciências Exatas e da Terra', 'Categoria'] = 'CET'
category_id_df.loc[category_id_df['Categoria'] == 'Ciências Humanas', 'Categoria'] = 'CH'
category_id_df.loc[category_id_df['Categoria'] == 'Ciências Sociais Aplicadas', 'Categoria'] = 'CSA'
category_id_df.loc[category_id_df['Categoria'] == 'Ciências da Saúde', 'Categoria'] = 'CS'
category_id_df.loc[category_id_df['Categoria'] == 'Engenharias', 'Categoria'] = 'E'
category_id_df.loc[category_id_df['Categoria'] == 'Lingüística, Letras e Artes', 'Categoria'] = 'LLA'
category_id_df.loc[category_id_df['Categoria'] == 'Multidisciplinar', 'Categoria'] = 'M'

#matriz de confusão
conf_mat = confusion_matrix(y_test, y_pred)
conf_mat.diagonal()/conf_mat.sum(axis=0)
fig, ax = plt.subplots(figsize=(5,5))
sns.heatmap(conf_mat, annot=True, fmt='d',
xticklabels=category_id_df.Categoria.values, yticklabels=category_id_df.Categoria.values)
plt.ylabel('Real')
plt.xlabel('Previsto')
plt.show()

In [ ]:
#imprime as métricas
print(metrics.classification_report(y_test, y_pred, target_names=df['Categoria'].unique()))

## Curvas-ROC

In [ ]:
from sklearn.svm import SVC

modelo_calibrado = CalibratedClassifierCV(LinearSVC(random_state=0), cv=10, ensemble=False)

gera_curvas_roc(modelo_calibrado, 63)
gera_curvas_roc(MultinomialNB(), 82)
gera_curvas_roc(LogisticRegression(random_state=0, solver='liblinear'), 82)

## Box-Plots

In [ ]:
df_resultados = read_ods("resultados-folds-formatado.ods",1)

#Define os tipos de dados das colunas
df_resultados['Classificador'] = df_resultados['Classificador'].astype(str)
df_resultados['Método'] = df_resultados['Método'].astype(str)
df_resultados['Atributo'] = df_resultados['Atributo'].astype(str)
df_resultados['Fold'] = df_resultados['Fold'].astype(int)
df_resultados['Grupo'] = df_resultados['Grupo'].astype(str)
df_resultados['Acurácia'] = df_resultados['Acurácia'].astype(float)

In [ ]:
df_titulo_acc = df_resultados[(df_resultados['Atributo'] == 'Título') & (df_resultados['Método'] == 'ACC')]

In [ ]:
def setBoxColors(bp, cor):
    for item in ['whiskers', 'fliers', 'medians', 'caps']:
        plt.setp(bp[item], color='black')
    plt.setp(bp["boxes"], facecolor=cor)

In [ ]:
def cria_box_plots(df, atributo, metodo):
    
    flierprops = dict(marker='*', markerfacecolor='black', markersize=10,
                  markeredgecolor='none')
    medianprops = dict(linestyle='-', linewidth=1.1)
    
    fig = plt.figure(figsize=(20,10))
    ax = plt.axes()
    
    classificadores = ['MVS','NBM','RL']
    posicoes = [[1,2,3,4,5],[8,9,10,11,12],[15,16,17,18,19]]
    grupos_ruido = ['C','I','II','III','IV']
    
    for i in range(len(classificadores)):
        lista_ruidos = []
        for j in range(len(grupos_ruido)):
            df_ruido = pd.DataFrame(df[(df['Grupo'] == grupos_ruido[j]) 
                                                          & (df['Classificador'] == classificadores[i])
                                                          & (df['Atributo'] == atributo) 
                                                          & (df['Método'] == metodo)])
            lista_ruidos.append(df_ruido['Acurácia'])
        
        bp = boxplot(lista_ruidos, positions=posicoes[i], patch_artist=True, flierprops=flierprops,
                    medianprops=medianprops, showcaps=False, widths=0.6, labels=['C', 'I', 'II', 'III', 'IV'])
        if i == 0:
            setBoxColors(bp,'blue')
        elif i == 1:
            setBoxColors(bp,'orange')
        elif i == 2:
            setBoxColors(bp,'green')
    
    xlim(0,20)
    ylim(25,65)
    #ax.set_xticks([1.5, 4.5, 7.5])
    ax.set_ylabel('Acurácia (%)')
    ax.set_yticks([25,30,35,40,45,50,55,60,65])
    
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    secax = ax.secondary_xaxis(-0.08)
    secax.xlim = (0,20)
    secax.spines['bottom'].set_visible(False)
    
    secax.set_xticks([3,10,17])
    
    secax.set_xticklabels(classificadores)
    secax.tick_params(axis='x', length=0)
    
    plt.yscale('linear')
    
    plt.title('Atributo {}'.format(atributo), y=1.05, pad=-10, loc='right')
    
    plt.plot([], c='blue')
    plt.plot([], c='orange')
    plt.plot([], c='green')

    

In [ ]:
cria_box_plots(df_resultados, 'Título', 'ACC')

In [ ]:
atributos = ('Título', 'Resumo', 'Palavras-chave', 'Texto')

for atributo in atributos:
    cria_box_plots(df_resultados, atributo, 'ACC')

In [ ]:


fig = plt.figure(figsize=(20,10))
ax = plt.axes()

df_ruidoC = pd.DataFrame(df_titulo_acc[(df_titulo_acc['Grupo'] == 'C') & (df_titulo_acc['Classificador'] == 'MVS')], columns=['Acurácia'])
df_ruidoI = pd.DataFrame(df_titulo_acc[(df_titulo_acc['Grupo'] == 'I') & (df_titulo_acc['Classificador'] == 'MVS')], columns=['Acurácia'])
df_ruidoII = pd.DataFrame(df_titulo_acc[(df_titulo_acc['Grupo'] == 'II') & (df_titulo_acc['Classificador'] == 'MVS')], columns=['Acurácia'])
df_ruidoIII = pd.DataFrame(df_titulo_ac[(df_titulo_acc['Grupo'] == 'III') & (df_titulo_acc['Classificador'] == 'MVS')], columns=['Acurácia'])
df_ruidoIV = pd.DataFrame(df_titulo_acc[(df_titulo_acc['Grupo'] == 'IV') & (df_titulo_acc['Classificador'] == 'MVS')], columns=['Acurácia'])

lista_ruidos_titulo_acc_mvs = [df_ruidoC['Acurácia'], df_ruidoI['Acurácia'], df_ruidoII['Acurácia'], df_ruidoIII['Acurácia'],
               df_ruidoIV['Acurácia']]

df_ruidoC = pd.DataFrame(df_titulo_acc[(df_titulo_acc['Grupo'] == 'C') & (df_titulo_acc['Classificador'] == 'NBM')], columns=['Acurácia'])
df_ruidoI = pd.DataFrame(df_titulo_acc[(df_titulo_acc['Grupo'] == 'I') & (df_titulo_acc['Classificador'] == 'NBM')], columns=['Acurácia'])
df_ruidoII = pd.DataFrame(df_titulo_acc[(df_titulo_acc['Grupo'] == 'II') & (df_titulo_acc['Classificador'] == 'NBM')], columns=['Acurácia'])
df_ruidoIII = pd.DataFrame(df_titulo_acc[(df_titulo_acc['Grupo'] == 'III') & (df_titulo_acc['Classificador'] == 'NBM')], columns=['Acurácia'])
df_ruidoIV = pd.DataFrame(df_titulo_acc[(df_titulo_acc['Grupo'] == 'IV') & (df_titulo_acc['Classificador'] == 'NBM')], columns=['Acurácia'])

lista_ruidos_titulo_acc_nbm = [df_ruidoC['Acurácia'], df_ruidoI['Acurácia'], df_ruidoII['Acurácia'], df_ruidoIII['Acurácia'],
               df_ruidoIV['Acurácia']]

df_ruidoC = pd.DataFrame(df_titulo_acc[(df_titulo_acc['Grupo'] == 'C') & (df_titulo_acc['Classificador'] == 'RL')], columns=['Acurácia'])
df_ruidoI = pd.DataFrame(df_titulo_acc[(df_titulo_acc['Grupo'] == 'I') & (df_titulo_acc['Classificador'] == 'RL')], columns=['Acurácia'])
df_ruidoII = pd.DataFrame(df_titulo_acc[(df_titulo_acc['Grupo'] == 'II') & (df_titulo_acc['Classificador'] == 'RL')], columns=['Acurácia'])
df_ruidoIII = pd.DataFrame(df_titulo_acc[(df_titulo_acc['Grupo'] == 'III') & (df_titulo_acc['Classificador'] == 'RL')], columns=['Acurácia'])
df_ruidoIV = pd.DataFrame(df_titulo_acc[(df_titulo_acc['Grupo'] == 'IV') & (df_titulo_acc['Classificador'] == 'RL')], columns=['Acurácia'])

lista_ruidos_titulo_acc_rl = [df_ruidoC['Acurácia'], df_ruidoI['Acurácia'], df_ruidoII['Acurácia'], df_ruidoIII['Acurácia'],
               df_ruidoIV['Acurácia']]

bp = boxplot(lista_ruidos_titulo_acc_mvs, positions=[1,2,3,4,5], patch_artist=True)
setBoxColors(bp,'blue')

bp = boxplot(lista_ruidos_titulo_acc_nbm, positions=[6,7,8,9,10], patch_artist=True)
setBoxColors(bp,'orange')


bp = boxplot(lista_ruidos_titulo_acc_rl, positions=[11,12,13,14,15], patch_artist=True)
setBoxColors(bp,'green')


xlim(0,16)
ylim(30,50)
ax.set_xticklabels(['C', 'I', 'II', 'III', 'IV','C', 'I', 'II', 'III', 'IV','C', 'I', 'II', 'III', 'IV'])
#ax.set_xticks([1.5, 4.5, 7.5])

plt.plot([], c='blue', label='MVS')
plt.plot([], c='orange', label='NBM')
plt.plot([], c='green', label='RL')
plt.legend()


In [ ]:
df_pc_acc = df_resultados[(df_resultados['Atributo'] == 'Palavras-chave') & (df_resultados['Método'] == 'ACC')]

In [ ]:
from pylab import plot, show, savefig, xlim, figure, ylim, legend, boxplot, setp, axes

fig = plt.figure(figsize=(20,10))
ax = plt.axes()

df_ruidoC = pd.DataFrame(df_titulo_acc[(df_titulo_acc['Grupo'] == 'C') & (df_titulo_acc['Classificador'] == 'MVS')], columns=['Acurácia'])
df_ruidoI = pd.DataFrame(df_titulo_acc[(df_titulo_acc['Grupo'] == 'I') & (df_titulo_acc['Classificador'] == 'MVS')], columns=['Acurácia'])
df_ruidoII = pd.DataFrame(df_titulo_acc[(df_titulo_acc['Grupo'] == 'II') & (df_titulo_acc['Classificador'] == 'MVS')], columns=['Acurácia'])
df_ruidoIII = pd.DataFrame(df_titulo_acc[(df_titulo_acc['Grupo'] == 'III') & (df_titulo_acc['Classificador'] == 'MVS')], columns=['Acurácia'])
df_ruidoIV = pd.DataFrame(df_titulo_acc[(df_titulo_acc['Grupo'] == 'IV') & (df_titulo_acc['Classificador'] == 'MVS')], columns=['Acurácia'])

lista_ruidos_titulo_acc_mvs = [df_ruidoC['Acurácia'], df_ruidoI['Acurácia'], df_ruidoII['Acurácia'], df_ruidoIII['Acurácia'],
               df_ruidoIV['Acurácia']]

df_ruidoC = pd.DataFrame(df_titulo_acc[(df_titulo_acc['Grupo'] == 'C') & (df_titulo_acc['Classificador'] == 'NBM')], columns=['Acurácia'])
df_ruidoI = pd.DataFrame(df_titulo_acc[(df_titulo_acc['Grupo'] == 'I') & (df_titulo_acc['Classificador'] == 'NBM')], columns=['Acurácia'])
df_ruidoII = pd.DataFrame(df_titulo_acc[(df_titulo_acc['Grupo'] == 'II') & (df_titulo_acc['Classificador'] == 'NBM')], columns=['Acurácia'])
df_ruidoIII = pd.DataFrame(df_titulo_acc[(df_titulo_acc['Grupo'] == 'III') & (df_titulo_acc['Classificador'] == 'NBM')], columns=['Acurácia'])
df_ruidoIV = pd.DataFrame(df_titulo_acc[(df_titulo_acc['Grupo'] == 'IV') & (df_titulo_acc['Classificador'] == 'NBM')], columns=['Acurácia'])

lista_ruidos_titulo_acc_nbm = [df_ruidoC['Acurácia'], df_ruidoI['Acurácia'], df_ruidoII['Acurácia'], df_ruidoIII['Acurácia'],
               df_ruidoIV['Acurácia']]

df_ruidoC = pd.DataFrame(df_titulo_acc[(df_titulo_acc['Grupo'] == 'C') & (df_titulo_acc['Classificador'] == 'RL')], columns=['Acurácia'])
df_ruidoI = pd.DataFrame(df_titulo_acc[(df_titulo_acc['Grupo'] == 'I') & (df_titulo_acc['Classificador'] == 'RL')], columns=['Acurácia'])
df_ruidoII = pd.DataFrame(df_titulo_acc[(df_titulo_acc['Grupo'] == 'II') & (df_titulo_acc['Classificador'] == 'RL')], columns=['Acurácia'])
df_ruidoIII = pd.DataFrame(df_titulo_acc[(df_titulo_acc['Grupo'] == 'III') & (df_titulo_acc['Classificador'] == 'RL')], columns=['Acurácia'])
df_ruidoIV = pd.DataFrame(df_titulo_acc[(df_titulo_acc['Grupo'] == 'IV') & (df_titulo_acc['Classificador'] == 'RL')], columns=['Acurácia'])

lista_ruidos_titulo_acc_rl = [df_ruidoC['Acurácia'], df_ruidoI['Acurácia'], df_ruidoII['Acurácia'], df_ruidoIII['Acurácia'],
               df_ruidoIV['Acurácia']]


bp = boxplot(lista_ruidos_acc_mvs, positions=[1,2,3,4,5], patch_artist=True)
setBoxColors(bp,'blue')

bp = boxplot(lista_ruidos_acc_nbm, positions=[6,7,8,9,10], patch_artist=True)
setBoxColors(bp,'orange')


bp = boxplot(lista_ruidos_acc_rl, positions=[11,12,13,14,15], patch_artist=True)
setBoxColors(bp,'green')


xlim(0,16)
ylim(20,70)
ax.set_xticklabels(['C', 'I', 'II', 'III', 'IV','C', 'I', 'II', 'III', 'IV','C', 'I', 'II', 'III', 'IV'])
#ax.set_xticks([1.5, 4.5, 7.5])

plt.plot([], c='blue', label='MVS')
plt.plot([], c='orange', label='NBM')
plt.plot([], c='green', label='RL')
plt.legend()
